In [2]:
import os
os.chdir(r"c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination")

import numpy as np
import pandas as pd
import torch
import transformers
from datasets import Dataset, load_dataset, load_from_disk, load_metric
from sklearn.metrics import mean_squared_error
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer

c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox\venv\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [24]:
df = pd.read_csv("data/cleaned_data/bulletins_w_labels_and_content.csv", dtype={'ID': object})  
df["label"] = round(df["Male"] / (df["Male"] +  df["Female"]), 3)
df = df[["Text", "label"]]
df

,ID,Job Description,Apps Received,Female,Male,Unknown_Gender,File Names,Label 60/40,Numeric label 60/40,Label 70/30,Numeric label 70/30,Cleaned text,Text,label
0,9206,311 DIRECTOR,54,20,31,3,311 DIRECTOR 9206 041814.txt,M,2,N,0,director class code open date annual salary du...,311 DIRECTOR Class Code: 9206 Open Date:...,0.608
1,1223,ACCOUNTING CLERK,648,488,152,8,ACCOUNTING CLERK 1223 071318.txt,W,1,W,1,accounting clerk class code open date exam ope...,ACCOUNTING CLERK Class Code: 1223 Open ...,0.238
2,7260,AIRPORT MANAGER,51,13,37,1,AIRPORT MANAGER 7260 120216.txt,M,2,M,2,airport manager class code open date exam open...,AIRPORT MANAGER Class Code: 7260 Open D...,0.740
3,3227,AIRPORT POLICE LIEUTENANT,48,9,38,1,AIRPORT POLICE LIEUTENANT 3227 091616.txt,M,2,M,2,airport police lieutenant class code open date...,AIRPORT POLICE LIEUTENANT ...,0.809
4,2400,AQUARIST,40,15,24,1,AQUARIST 2400 050214.txt,M,2,N,0,aquarist class code open date annual salary ca...,AQUARIST Class Code: 2400 Open Date: 05...,0.615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
172,7840,WASTEWATER TREATMENT LABORATORY MANAGER,16,6,9,1,WASTEWATER TREATMENT LABORATORY MANAGER 7840 1...,M,2,N,0,wastewater treatment laboratory manager class ...,WASTEWATER TREATMENT LABORATORY MANAGER Class...,0.600
173,4123,WASTEWATER TREATMENT OPERATOR,125,9,113,3,WASTEWATER TREATMENT OPERATOR 120718.txt,M,2,M,2,wastewater treatment operator class code open ...,WASTEWATER TREATMENT OPERATOR Class Code: ...,0.926
174,7857,WATER MICROBIOLOGIST,179,89,82,8,WATER MICROBIOLOGIST 7857 072514 rev073114.txt,N,0,N,0,water microbiologist class code open date revi...,WATER MICROBIOLOGIST Class Code: 7857...,0.480
175,3912,WATER UTILITY WORKER,96,2,92,2,WATER UTILITY WORKER 3912 120817.txt,M,2,M,2,water utility worker class code open date exam...,WATER UTILITY WORKER Class Code: 3912 Op...,0.979


In [16]:
dataset = Dataset.from_pandas(df, preserve_index=False) 
dataset = dataset.train_test_split(test_size=0.3) 
dataset

DatasetDict({
    train: Dataset({
        features: ['Text', 'label'],
        num_rows: 123
    })
    test: Dataset({
        features: ['Text', 'label'],
        num_rows: 54
    })
})

In [98]:
text_beginnings_test = []
labels_test = []
for i in range(len(dataset["test"])):
    text_beginnings_test.append(dataset["test"][i]["Text"][:70])
    labels_test.append(dataset["test"][i]["label"])

In [103]:
job_descriptions_test = []
count = 0
job_descriptions = list(df["Job Description"])
for job in job_descriptions:
    for text in text_beginnings_test:
        if job in text:
            job_descriptions_test.append(job)

In [104]:
job_descriptions_test = list(set(job_descriptions_test))
job_descriptions_test = sorted(job_descriptions_test)
job_descriptions_test.remove("BUILDING MECHANICAL INSPECTOR")
job_descriptions_test.remove("ENGINEERING GEOLOGIST")
job_descriptions_test.remove("ENVIRONMENTAL ENGINEER")
job_descriptions_test.remove("PORT POLICE LIEUTENANT")
job_descriptions_test.remove("SENIOR ELECTRICAL MECHANIC")
job_descriptions_test.remove("UTILITY BUYER")

In [105]:
text_beginnings_test = [text.strip() for text in text_beginnings_test]
text_beginnings_test = sorted(text_beginnings_test)
for job, text in list(zip(job_descriptions_test, text_beginnings_test)):
    print(job, "\t\t\t\t", text)

AIRPORT POLICE LIEUTENANT 				 AIRPORT POLICE LIEUTENANT
AQUARIST 				 AQUARIST Class Code:       2400 Open Date:  05-02-14 ANNUAL SALARY  $5
ARTS MANAGER 				 ARTS MANAGER Class Code:      2455 Open Date: 09-28-18 (Exam Open to A
ASSISTANT UTILITY BUYER 				 ASSISTANT UTILITY BUYER  Class Code:       1860 Open Date:  05-18-18 (
BUILDING MAINTENANCE DISTRICT SUPERVISOR 				 BUILDING MAINTENANCE DISTRICT SUPERVISOR   Class Code:       3190  Ope
BUILDING REPAIR SUPERVISOR 				 BUILDING REPAIR SUPERVISOR  								Class Code:       3338
CHIEF CLERK PERSONNEL 				 CHIEF CLERK PERSONNEL  Class Code:       1260       Open Date:  04-21-
CHIEF FINANCIAL OFFICER 				 CHIEF FINANCIAL OFFICER
CHIEF OF AIRPORT PLANNING 				 CHIEF OF AIRPORT PLANNING  Class Code:       7945 Open Date:  3-24-17
CHIEF OF DRAFTING OPERATIONS 				 CHIEF OF DRAFTING OPERATIONS  Class Code:       7271 Open Date:  04-20
COMMERCIAL SERVICE SUPERVISOR 				 COMMERCIAL SERVICE SUPERVISOR  												Class Code:       1213
COM

In [106]:
# Gå igenom job_descriptions_test och lägga motsvarande cleaned_text i en lista som X för att kunna jämföra resultaten med CatBoost och LogReg

In [17]:
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

loading configuration file config.json from cache at C:\Users\britt/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading file vocab.txt from cache at C:\Users\britt/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\vocab.txt
loading file tokenizer.json from cache at C:\Users\britt/.cache\huggingface\hub\models--dis

In [54]:
len(dataset["test"])

54

In [18]:
def tokenize_function(examples):
    return tokenizer(examples["Text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

100%|██████████| 1/1 [00:00<00:00, 15.25ba/s]


In [19]:
model = AutoModelForSequenceClassification.from_pretrained("distilbert-base-uncased", num_labels=1)

loading configuration file config.json from cache at C:\Users\britt/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\config.json
Model config DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at C:\Users\britt/.cache\huggingface\hub\models--distilbert-base-uncased\snapshots\1c4513b2eedbda136f57676a34eea67aba266e5c\pytorch

In [20]:
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    rmse = mean_squared_error(labels, predictions, squared=False)
    return {"rmse": rmse}

In [21]:
training_args = TrainingArguments(output_dir="test_trainer",
                                  logging_strategy="epoch",
                                  evaluation_strategy="epoch",
                                  per_device_train_batch_size=16,
                                  per_device_eval_batch_size=16,
                                  num_train_epochs=3,
                                  save_total_limit = 2,
                                  save_strategy = 'no',
                                  load_best_model_at_end=False
                                  )
trainer = Trainer(model=model,
                  args=training_args,
                  train_dataset=tokenized_datasets["train"],
                  eval_dataset=tokenized_datasets["test"],
                  compute_metrics=compute_metrics
)
trainer.train()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
  0%|          | 0/24 [01:41<?, ?it/s]
The following columns in the training set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Text. If Text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
c:\Users\britt\Desktop\YH\Applicerad AI\job_discrimination_sandbox\venv\lib\site-packages\transformers\optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
***** Running training *****
  Num exa

{'loss': 0.1534, 'learning_rate': 3.3333333333333335e-05, 'epoch': 1.0}


                                              
 33%|███▎      | 8/24 [04:04<07:07, 26.70s/it]

{'eval_loss': 0.05538410693407059, 'eval_rmse': 0.2353382706642151, 'eval_runtime': 29.5933, 'eval_samples_per_second': 1.825, 'eval_steps_per_second': 0.135, 'epoch': 1.0}


 67%|██████▋   | 16/24 [07:57<03:43, 27.99s/it]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Text. If Text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54
  Batch size = 16


...

100%|██████████| 24/24 [15:16<00:00, 39.36s/it]The following columns in the evaluation set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: Text. If Text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Evaluation *****
  Num examples = 54
  Batch size = 16


{'loss': 0.0353, 'learning_rate': 0.0, 'epoch': 3.0}


                                               
100%|██████████| 24/24 [15:42<00:00, 39.36s/it]

Training completed. Do not forget to share your model on huggingface.co/models =)


100%|██████████| 24/24 [15:42<00:00, 39.28s/it]

{'eval_loss': 0.03049021027982235, 'eval_rmse': 0.17461444437503815, 'eval_runtime': 25.867, 'eval_samples_per_second': 2.088, 'eval_steps_per_second': 0.155, 'epoch': 3.0}
{'train_runtime': 942.6503, 'train_samples_per_second': 0.391, 'train_steps_per_second': 0.025, 'train_loss': 0.07943640152613322, 'epoch': 3.0}


TrainOutput(global_step=24, training_loss=0.07943640152613322, metrics={'train_runtime': 942.6503, 'train_samples_per_second': 0.391, 'train_steps_per_second': 0.025, 'train_loss': 0.07943640152613322, 'epoch': 3.0})

In [22]:
# save the model/tokenizer
model.save_pretrained("model")
tokenizer.save_pretrained("tokenizer")
# load the model/tokenizer
from transformers import AutoModelForTokenClassification
model = AutoModelForSequenceClassification.from_pretrained("model")
tokenizer = AutoTokenizer.from_pretrained("tokenizer")

Configuration saved in model\config.json
Model weights saved in model\pytorch_model.bin
tokenizer config file saved in tokenizer\tokenizer_config.json
Special tokens file saved in tokenizer\special_tokens_map.json
loading configuration file model\config.json
Model config DistilBertConfig {
  "_name_or_path": "model",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "regression",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transformers_version": "4.25.1",
  "vocab_size": 30522
}

loading weights file model\pytorch_model.bin
All mod

In [23]:
#Using the model on new texts

trainer = Trainer(model=model)

def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True) 

def pipeline_prediction(text):
    df= pd.DataFrame({'text':[text]})
    dataset = Dataset.from_pandas(df, preserve_index=False) 
    tokenized_datasets = dataset.map(tokenize_function)
    raw_pred, _, _ = trainer.predict(tokenized_datasets) 
    return(raw_pred[0][0])

No `TrainingArguments` passed, using `output_dir=tmp_trainer`.
PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [32]:
texts = list(df["Text"])
for i in range(5):
    print(pipeline_prediction(texts[i]))

100%|██████████| 1/1 [00:00<00:00, 131.05ex/s]
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8
100%|██████████| 1/1 [00:00<?, ?it/s]


0.5478056


100%|██████████| 1/1 [00:00<00:00, 100.89ex/s]
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8
100%|██████████| 1/1 [00:00<?, ?it/s]


0.46636376


100%|██████████| 1/1 [00:00<00:00, 139.10ex/s]
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8
100%|██████████| 1/1 [00:00<?, ?it/s]


0.68880635


100%|██████████| 1/1 [00:00<00:00, 70.76ex/s]
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8
100%|██████████| 1/1 [00:00<?, ?it/s]


0.73103267


100%|██████████| 1/1 [00:00<00:00, 112.70ex/s]
The following columns in the test set don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: text. If text are not expected by `DistilBertForSequenceClassification.forward`,  you can safely ignore this message.
***** Running Prediction *****
  Num examples = 1
  Batch size = 8
100%|██████████| 1/1 [00:00<?, ?it/s]

0.78612345
